In [44]:
#function of 
def lorz(t,x):
    xdot=np.zeros(3).reshape(3,1)
    sig=10.0
    rho=28.0
    bet=8.0/3.0
    xdot[0]=sig*(x[1]-x[0])
    xdot[1] = rho*x[0]-x[1]-x[0]*x[2]
    xdot[2] = x[0]*x[1]-bet*x[2]
    return (xdot)

In [5]:
import pandas as pd
df = pd.read_excel(r'C:\Users\INFO-DSK-02\Desktop\Lorentz\Lorentz_Requirements_04032020\Lyapunov Exponent J. C. Sprott\Input_Data\lorentz_noise_data_1.xlsx')

In [41]:
x = df[['x']]

In [64]:
from scipy.integrate import ode


import numpy as np
import matplotlib.pyplot as plt
def lyapunov(funfuc,st, kkmax, x):
    funfun = funfuc
    n=x.shape[1]
    print(n)
    ex=np.zeros((n,1))
    sum_=np.zeros((n,1))
    u=np.identity(n)
    print(u.shape)
    tinit1=0
    for kindex in range(0,kkmax):
        xx=[]
        tfinal=tinit1+st
        print(tfinal)
        rs=np.reshape(u,(n*n, 1))
        print(rs.shape)
        print('x', x.shape)
        var=np.concatenate((x,rs), axis=0)
        print(var.shape)
        #xx=ode(funfnc,[tinit,tfinal],var)
        r=ode(lorz).set_integrator('vode', method='adams',
                           rtol =1.0e-7,
                          with_jacobian=True)
        r.set_initial_value(tinit1)
        while r.successful() and r.t <= tfinal:
            r.integrate(r.t + st)
            xx.append(r.y)  
            t.append(r.t)
        
        ss1=xx.shape[0]
        ss2=xx.shape[1]
        var=xx[ss1,].conj().transpose()
        
        x=var[0:n]
        dx=np.reshape(var[n:len(var)],(n,n))
        for i in range(0,n):
            v[:,i]=dx[:,i]
            for j in range(0,i-1):
                dotp=v[:,i].conj().transpose()
                v[:,i]=v[:,i]-dotp*u[:,j]
            veclen=v[:,i].conj().transpose()*v[:,i]
            veclen=math.sqrt(veclen)
            u[:,i]=v[:,i]/veclen
            sum_[i]=sum_[i]+log(veclen)
            ex[kindex,i]=sum_[i]/(kindex*st)
        it[kindex]=tfinal
        tinit=tfinal
    plt.plot(it,ex)
    siex=ex.shape
    lyap = ex[siex[1],:]
    return lyap

            
                
                
            
            
        

            
        

In [54]:
x.shape

(2829, 1)

In [66]:
lyapunov(lorz,1,100,x)

1
(1, 1)
1
(1, 1)
x (2829, 1)
(2830, 1)


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
veclen=v(:,i)'*v(:,i);
veclen=sqrt(veclen);
u(:,i)=v(:,i)/veclen;

In [ ]:
for j=1:i-1
        dotp=v(:,i)'*u(:,j);
        v(:,i)=v(:,i)-dotp*u(:,j);

In [ ]:
dx=reshape(var(n+1:length(var)),n,n);

In [5]:
n=4
u=np.identity(n)

np.reshape(u,(n*n, 1))

array([[1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.]])

In [ ]:
var=[x;reshape(u,n*n,1)]

In [ ]:
ex=zeros(n,1);
sum=zeros(n,1);

In [ ]:
%Calculate largest Lyapunov exponent from ODE directly.
%Algorithm is based on Alan Wolf, 1985.

%By Xiaowei Huai
%2015/5/28
%------------------------------------------------------------

%Spin-up to accquire post-transient initial condition

from scipy.integrate import ode



[~,yspin] = ode45(@lorenz63,1:0.01:50,[1,1,1]);



yinit = yspin(length(yspin),:);
orthmatrix = eye(3);


% orthmatrix = [1 0 0;
%               0 1 0;
%               0 0 1];
y = zeros(12,1);
y(1:3) = yinit;
y(4:12) = orthmatrix;

tstart = 0; % 时间初始值
%tstep = 1e-6; % 时间步长
%wholetimes = 1e6; % 总的循环次数
%steps = 1000; % 每次演化的步数
%iteratetimes = wholetimes/steps; % 演化的次数
iteratetimes = 500;
tincre = 1.0;
sum = zeros(3,1);

% 初始化三个Lyapunov指数
expo = zeros(iteratetimes,3);

for i=1:iteratetimes
    %tend = tstart + tstep*steps;
    %tspan = tstart:tstep:tend;   
    tend = tstart + tincre;
    [~,Y] = ode45(@lorenz_ode,[tstart,tend], y);

    % 取积分得到的最后一个时刻的值
    y = Y(size(Y,1),:);
    % 重新定义起始时刻
    tstart = tend;
    y0 = [y(4) y(7) y(10);
          y(5) y(8) y(11);
          y(6) y(9) y(12)];
    %正交化
    [y0,znorm] = GS(y0);
    sum = sum + log(znorm);
    y(4:12) = y0;

    %三个Lyapunov指数
    for j=1:3
        expo(i,j) = sum(j)/tstart;
    end
end

lyap = expo(length(expo),:);
disp(lyap)
%In this frame, Lyapunov exponents will be  0.9065   -0.0023  -14.5036.

%   作Lyapunov指数谱图
i = 1:iteratetimes;
plot(i,expo(:,1),'r-',i,expo(:,2),'g-',i,expo(:,3),'b-','LineWidth',1.5)
xlabel('\fontsize{14}Time');ylabel('\fontsize{14}Lyapunov Exponents')
legend('\lambda_1','\lambda_2','\lambda_3','Location','Best')
legend('boxoff')
print(gcf,'-dpng','LyapExpoSpectrum.png')